Download the amazon product dataset from [here](http://jmcauley.ucsd.edu/data/amazon/)

In [5]:
import json

def get_reviews():
    for line in open("/Users/plee/workspace/ipython_notebooks/data/Apps_for_Android_5.json", "rb").readlines():
        js = json.loads(line)    
        yield js['reviewText'], js['overall']

I am just taking the reviews with less than 75 words

In [8]:
import numpy as np

SENTENCE_LENGTH = 75

reviews = []
scores = []
for review, score in get_reviews():
    if score != 3:
        word_count = len(review.split(' '))
        if word_count < SENTENCE_LENGTH:
            reviews.append(str(review))
            scores.append(score)

In [7]:
y = map(lambda x: 0 if x < 3 else 1, scores)

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class Vectorizer:
    def __init__(self, maxlen):
        self.tokenizer = Tokenizer()
        self.maxlen = maxlen
        
    def fit(self, texts):
        self.tokenizer.fit_on_texts(texts)
        self.word_index = self.tokenizer.word_index

    def transform(self, texts):
        sequences = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(sequences, maxlen=self.maxlen)

    def fit_transform(self, texts):
        self.fit(texts)
        return self.transform(texts)

In [98]:
# import numpy as np

# reviews = []
# scores = []
# for review, score in get_reviews():
#     if score != 3:
#         word_count = len(review.split(' '))
#         if word_count < 75:
#             reviews.append(str(review))
#             scores.append(score)
            
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# y = scaler.fit_transform(np.array(scores).reshape((len(scores),1)))




/Users/plee/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/plee/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [14]:
v = Vectorizer(SENTENCE_LENGTH)
X = v.fit_transform(reviews)
input_dim = len(v.word_index) + 1
del reviews


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Conv1D, MaxPooling1D, Dropout, Flatten, AveragePooling1D
dropout = 0.1
nb_filter = 4
nb_conv = 3
model = Sequential()
model.add(Embedding(input_dim, 32, input_length=SENTENCE_LENGTH))
model.add(Dropout(dropout))

model.add(Conv1D(nb_filter, nb_conv, padding='same'))

model.add(MaxPooling1D(pool_size=6, padding='same'))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 75, 32)            4245632   
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 75, 4)             388       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 4)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 52)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 53        
Total params: 4,246,073
Trainable params: 4,246,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

callbacks = []
callbacks.append(early_stopping_monitor)
history = model.fit(X, y, epochs=2, validation_split=0.2,
        callbacks=callbacks)

Train on 483284 samples, validate on 120822 samples
Epoch 1/2
206944/483284 [===========>..................] - ETA: 264s - loss: 0.2270 - acc: 0.9076 